# Alchemi quickstartThis notebook builds a tiny synthetic hyperspectral cube and walks through a few core APIs:* Creating a `Cube` from NumPy arrays.* Inspecting basic cube metadata and a pixel `Spectrum`.* Tokenising bands with `BandTokenizer`.* Comparing spectra with a simple spectral angle metric.The example uses only in-memory arrays so it runs quickly in a fresh `pip install -e .[dev]` environment.

In [ ]:
import numpy as npfrom alchemi.data import Cubefrom alchemi.tokens import BandTokenizer

In [ ]:
rng = np.random.default_rng(seed=7)height, width, band_count = 16, 16, 10wavelengths_nm = np.linspace(400.0, 700.0, band_count)baseline = 0.25 + 0.5 * np.sin(np.linspace(0, np.pi, band_count))noise = rng.normal(scale=0.02, size=(height, width, band_count))cube_data = np.clip(baseline + noise, 0.0, 1.0)cube = Cube(    data=cube_data,    axis=wavelengths_nm,    axis_unit="wavelength_nm",    value_kind="reflectance",    attrs={"sensor": "synthetic"},)cube

In [ ]:
print(f"Cube shape (H, W, bands): {cube.shape}")print(f"Band count: {cube.band_count}")print(f"Wavelength range (nm): {cube.axis.min():.1f} – {cube.axis.max():.1f}")print(f"Value range: {cube.data.min():.3f} – {cube.data.max():.3f}")

In [ ]:
# Inspect a single pixel spectrumsample = cube.sample_at(0, 0)spectrum = sample.spectrumprint(f"Sample sensor: {sample.meta.sensor_id}")print(f"Spectrum kind: {spectrum.kind.value}")print("First five wavelengths (nm):", np.round(spectrum.wavelengths.nm[:5], 1))print("First five reflectances:", np.round(spectrum.values[:5], 3))

In [ ]:
tokenizer = BandTokenizer()tokens = cube.to_tokens(tokenizer)print("Band token array shape:", tokens.bands.shape)print("Pooled token shape:", tokens.pooled.shape)print("Value normalisation:", tokens.meta.value_norm)print("Includes band width:", tokens.meta.include_width)

In [ ]:
def spectral_angle(a: np.ndarray, b: np.ndarray) -> float:    a = np.asarray(a, dtype=np.float64)    b = np.asarray(b, dtype=np.float64)    dot = float(np.sum(a * b))    denom = float(np.linalg.norm(a) * np.linalg.norm(b) + 1e-12)    return float(np.arccos(np.clip(dot / denom, -1.0, 1.0)))s1 = cube.sample_at(0, 0).spectrum.valuess2 = cube.sample_at(1, 1).spectrum.valuesangle_rad = spectral_angle(s1, s2)print(f"Spectral angle between (0, 0) and (1, 1): {np.degrees(angle_rad):.2f}°")